### Finetuning Model for Spanish-English translation using Ted Talk subtitles dataset

In [28]:
# Loggin to hub
from huggingface_hub import notebook_login
notebook_login()

Model, Dataset & Metric loading

In [37]:
model_checkpoint='Helsinki-NLP/opus-mt-en-mul'

In [38]:
from datasets import load_from_disk

In [39]:
data = load_from_disk("ted_trans_dataset")

In [40]:
data

DatasetDict({
    train: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 2454
    })
    validation: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 307
    })
    test: Dataset({
        features: ['Original_Sentence', 'Translate_SP', '__index_level_0__'],
        num_rows: 307
    })
})

In [7]:
# Carga métrica ROUGE
from datasets import load_metric
import evaluate

rouge_score = load_metric("rouge")

Tokenizer and Preprocessing

In [8]:
import transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [doc for doc in examples["Original_Sentence"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Translate_SP"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = data.map(preprocess_function, batched=True)

Loading cached processed dataset at ted_trans_dataset/train\cache-5d82fcb5ee8de5f6.arrow
Loading cached processed dataset at ted_trans_dataset/validation\cache-774e890d4668dad1.arrow
Loading cached processed dataset at ted_trans_dataset/test\cache-20596ff67f2124b7.arrow


In [11]:
# PT
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [12]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_checkpoint)

In [13]:
config

MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-mul",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      64109
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 64109,
  "decoder_vocab_size": 64110,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 64110,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Metrics function

In [15]:
import numpy as np
import nltk
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asr_l\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training Args

In [16]:
# PT

model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned_es_en_translator",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

Training

In [17]:
# PT

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: __index_level_0__, Original_Sentence, Translate_SP. If __index_level_0__, Original_Sentence, Translate_SP are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
c:\Users\asr_l\miniconda3\envs\translate_subs\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2454
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 614









































































































{'loss': 1.2981, 'learning_rate': 3.7133550488599353e-06, 'epoch': 0.81}


Model weights saved in opus-mt-en-mul-finetuned-xsum\checkpoint-500\pytorch_model.bin
tokenizer config file saved in opus-mt-en-mul-finetuned-xsum\checkpoint-500\tokenizer_config.json
Special tokens file saved in opus-mt-en-mul-finetuned-xsum\checkpoint-500\special_tokens_map.json


















































































































The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: __index_level_0__, Original_Sentence, Translate_SP. If __index_level_0__, Original_Sentence, Translate_SP are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 307
  Batch size = 4


























































































































































                                                   


         

{'eval_loss': 1.3484039306640625, 'eval_rouge1': 64.9473, 'eval_rouge2': 42.2411, 'eval_rougeL': 61.9713, 'eval_rougeLsum': 62.9319, 'eval_gen_len': 61.5505, 'eval_runtime': 1156.3348, 'eval_samples_per_second': 0.265, 'eval_steps_per_second': 0.067, 'epoch': 1.0}
{'train_runtime': 2859.8927, 'train_samples_per_second': 0.858, 'train_steps_per_second': 0.215, 'train_loss': 1.3345782252010383, 'epoch': 1.0}


TrainOutput(global_step=614, training_loss=1.3345782252010383, metrics={'train_runtime': 2859.8927, 'train_samples_per_second': 0.858, 'train_steps_per_second': 0.215, 'train_loss': 1.3345782252010383, 'epoch': 1.0})

Saving Model and Testing it from local disk

In [24]:
trainer.save_model('mod_translate')

Saving model checkpoint to mod_translate
Configuration saved in mod_translate\config.json
Model weights saved in mod_translate\pytorch_model.bin
tokenizer config file saved in mod_translate\tokenizer_config.json
Special tokens file saved in mod_translate\special_tokens_map.json


In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained('mod_translate')

loading configuration file mod_translate\config.json
Model config MarianConfig {
  "_name_or_path": "mod_translate",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      64109
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 64109,
  "decoder_vocab_size": 64110,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 64110,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABE

In [30]:
tokenizer = AutoTokenizer.from_pretrained('mod_translate')

Didn't find file mod_translate\target_vocab.json. We won't load it.
Didn't find file mod_translate\added_tokens.json. We won't load it.
loading file mod_translate\source.spm
loading file mod_translate\target.spm
loading file mod_translate\vocab.json
loading file None
loading file mod_translate\tokenizer_config.json
loading file None
loading file mod_translate\special_tokens_map.json


In [41]:
text = tokenizer(['Hi, trying this model out'],return_tensors="pt")

In [33]:
output_text = model.generate(text['input_ids'])

In [34]:
output = tokenizer.decode(output_text.squeeze(), skip_special_tokens=True)

In [24]:
output

'Hola, probando este modelo'

Pushing model to hub

In [ ]:
model_name = model_checkpoint.split("/")[-1]


In [42]:
#model.push_to_hub(f"{model_name}-finetuned_en_sp_translator")